<a href="https://colab.research.google.com/github/lukas-gysin/bverl-miniproject/blob/main/notebooks/07_Julian_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%capture
!pip install torchshow torchinfo lightning

In [24]:
%matplotlib inline
from io import BytesIO
import os
from pathlib import Path
import tempfile
from typing import Callable
import zipfile

import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import TensorBoardLogger
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchinfo
import torchmetrics
import torchshow
from torchvision import transforms
from torchvision.transforms import v2

In [25]:
L.seed_everything(1234)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using: {device}")

INFO: Seed set to 1234
INFO:lightning.fabric.utilities.seed:Seed set to 1234


Using: cuda


In [26]:

try:
  import geoguesser

  print("package installed, all good")
except ImportError as e:
  print("Installing from git repo")
  os.system("pip install git+https://github.com/lukas-gysin/bverl-miniproject")

from geoguesser.classifier import Classifier
from geoguesser.dataset import EuroSATDataset
from geoguesser.dataset import EuroSATDataLoader
from geoguesser.trainer import Trainer

# Third Party Libraries
import lightning as L
import torch
import torch.nn as nn
import torchmetrics


package installed, all good


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [27]:


datasetRGB = EuroSATDataset.download(dataset='RGB')
print(datasetRGB.classes)
# torchshow.show(datasetRGB[0]['image'])

Dataset already saved in /workspace/code/data/RGB.zip - not overwriting
Dataset already extracted to /workspace/code/data/RGB - not overwriting
['Forest', 'Highway', 'Industrial', 'River', 'PermanentCrop', 'SeaLake', 'Pasture', 'HerbaceousVegetation', 'Residential', 'AnnualCrop']


In [28]:
import torchvision.models as models

ResnetRGB = models.resnet18(pretrained=True)

tr_train = transforms.Compose(
    [
        # transforms.v2.RGB(),
        transforms.RandomResizedCrop((128, 128)),
        transforms.RandomHorizontalFlip(),
        # transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

tr_val = transforms.Compose(
    [
        # transforms.v2.RGB(),
        transforms.Resize((128, 128)),
        # transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)


print(torchinfo.summary(ResnetRGB, input_size=(1, 3, 64, 64)))


ResnetRGB.fc = nn.Sequential(nn.Linear(512, 10))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 32, 32]           9,408
├─BatchNorm2d: 1-2                       [1, 64, 32, 32]           128
├─ReLU: 1-3                              [1, 64, 32, 32]           --
├─MaxPool2d: 1-4                         [1, 64, 16, 16]           --
├─Sequential: 1-5                        [1, 64, 16, 16]           --
│    └─BasicBlock: 2-1                   [1, 64, 16, 16]           --
│    │    └─Conv2d: 3-1                  [1, 64, 16, 16]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 16, 16]           128
│    │    └─ReLU: 3-3                    [1, 64, 16, 16]           --
│    │    └─Conv2d: 3-4                  [1, 64, 16, 16]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 16, 16]           128
│    │    └─ReLU: 3-6                    [1, 64, 16, 16]           --
│

In [29]:
seed=1234

In [30]:

# ids = [i for i in range(0, len(datasetRGB.observations))]
# labels = [x["label"] for x in datasetRGB.observations]

# train_ids, test_ids = train_test_split(ids, train_size=0.8, stratify=labels)
# val_ids, test_ids = train_test_split(test_ids, test_size=0.5, stratify=[labels[i] for i in test_ids])

# dataset_train = EuroSATDataset.from_subset(datasetRGB, train_ids, tr_train)
# dataset_val = EuroSATDataset.from_subset(datasetRGB, val_ids, tr_val)
# dataset_test = EuroSATDataset.from_subset(datasetRGB, test_ids)

# dataloader_train = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=os.cpu_count())
# dataloader_val = DataLoader(dataset_val, batch_size=128, shuffle=True, num_workers=os.cpu_count())
# dataloader_test = DataLoader(dataset_test, batch_size=128, shuffle=True, num_workers=os.cpu_count())




dataloaders = EuroSATDataLoader(datasetRGB, seed=seed)
dataloader_test = dataloaders.testing()
dataloader_train = dataloaders.training(transformer=tr_train, batch_size=128)
dataloader_val = dataloaders.validation(transformer=tr_val, batch_size=128)

Created a subset with 2_700 of 27_000 images
Created a subset with 21_600 of 27_000 images
Created a subset with 2_700 of 27_000 images


In [31]:

model = Classifier(ResnetRGB, weight_decay=0, learning_rate=0.0001)
trainer = Trainer(seed)


# trainer = L.Trainer(
#     devices="auto",
#     accelerator="auto",
#     precision="32",
#     max_epochs=30,
#     # max_steps=150,
#     enable_checkpointing=False,
# )


trainer.fit(model, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

INFO: Seed set to 1234
INFO:lightning.fabric.utilities.seed:Seed set to 1234
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ResNet             | 11.2 M | train
1 | loss_fn        | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | train_loss     | MeanMetric         | 0      | train
4 | val_accuracy   | MulticlassAccuracy | 0      | train
5

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val/acc_epoch improved. New best score: 0.948
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/acc_epoch improved. New best score: 0.948


...

INFO: Metric val/acc_epoch improved by 0.001 >= min_delta = 0.0. New best score: 0.970
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/acc_epoch improved by 0.001 >= min_delta = 0.0. New best score: 0.970


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val/acc_epoch did not improve in the last 3 records. Best score: 0.970. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric val/acc_epoch did not improve in the last 3 records. Best score: 0.970. Signaling Trainer to stop.


In [ ]:
torch.save(model.state_dict(), "/content/gdrive/MyDrive/RGBResnet")

# MS

In [32]:
import torchvision.models as models

netMS = models.resnet18()

# 2. Replace the first conv layer
old_conv = netMS.conv1

# Create a new conv that has 13 input channels,
# but the same other parameters as the old conv
netMS.conv1 = nn.Conv2d(
    in_channels=13,
    out_channels=old_conv.out_channels,  # 64
    kernel_size=old_conv.kernel_size,    # (7, 7)
    stride=old_conv.stride,              # (2, 2)
    padding=old_conv.padding,            # (3, 3)
    bias=old_conv.bias is not None
)


print(torchinfo.summary(netMS, input_size=(1, 13, 64, 64)))


netMS.fc = nn.Linear(512, 10)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 1000]                 --
├─Conv2d: 1-1                            [1, 64, 32, 32]           40,768
├─BatchNorm2d: 1-2                       [1, 64, 32, 32]           128
├─ReLU: 1-3                              [1, 64, 32, 32]           --
├─MaxPool2d: 1-4                         [1, 64, 16, 16]           --
├─Sequential: 1-5                        [1, 64, 16, 16]           --
│    └─BasicBlock: 2-1                   [1, 64, 16, 16]           --
│    │    └─Conv2d: 3-1                  [1, 64, 16, 16]           36,864
│    │    └─BatchNorm2d: 3-2             [1, 64, 16, 16]           128
│    │    └─ReLU: 3-3                    [1, 64, 16, 16]           --
│    │    └─Conv2d: 3-4                  [1, 64, 16, 16]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 16, 16]           128
│    │    └─ReLU: 3-6                    [1, 64, 16, 16]           --


In [33]:
datasetMS = EuroSATDataset.download(dataset='MS')
print(datasetMS.classes)
print(datasetMS[0])
# torchshow.show(datasetMS[0]['image'][[3,2,1]])

Dataset already saved in /workspace/code/data/MS.zip - not overwriting
Dataset already extracted to /workspace/code/data/MS - not overwriting
['Forest', 'Highway', 'Industrial', 'River', 'PermanentCrop', 'SeaLake', 'Pasture', 'HerbaceousVegetation', 'Residential', 'AnnualCrop']
{'image': tensor([[[0.0571, 0.0571, 0.0286,  ..., 0.1143, 0.1429, 0.1429],
         [0.0571, 0.0571, 0.0286,  ..., 0.1143, 0.1429, 0.1429],
         [0.0571, 0.0571, 0.0571,  ..., 0.1143, 0.1429, 0.1714],
         ...,
         [0.7143, 0.7143, 0.7143,  ..., 0.4286, 0.4286, 0.4286],
         [0.7143, 0.7143, 0.7143,  ..., 0.4571, 0.4571, 0.4286],
         [0.7143, 0.7143, 0.7143,  ..., 0.4571, 0.4571, 0.4571]],

        [[0.1451, 0.1451, 0.1325,  ..., 0.1893, 0.2555, 0.2429],
         [0.1451, 0.1451, 0.1325,  ..., 0.1893, 0.2555, 0.2429],
         [0.1609, 0.1609, 0.1514,  ..., 0.2429, 0.2808, 0.3028],
         ...,
         [0.2808, 0.2808, 0.2587,  ..., 0.1924, 0.2050, 0.2177],
         [0.2871, 0.2871, 0.274

In [34]:
dataloadersMS = EuroSATDataLoader(datasetMS, seed=seed)
dataloader_testMS = dataloadersMS.testing()
dataloader_trainMS = dataloadersMS.training(batch_size=64)
dataloader_valMS = dataloadersMS.validation(batch_size=64)

Created a subset with 2_700 of 27_000 images
Created a subset with 21_600 of 27_000 images
Created a subset with 2_700 of 27_000 images


In [40]:

modelms = Classifier(netMS, learning_rate=0.0001)
trainer = Trainer(seed)

trainer.fit(modelms, train_dataloaders=dataloader_trainMS, val_dataloaders=dataloader_valMS)

INFO: Seed set to 1234
INFO:lightning.fabric.utilities.seed:Seed set to 1234
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ResNet             | 11.2 M | train
1 | loss_fn        | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | train_loss     | MeanMetric         | 0      | train
4 | val_accuracy   | MulticlassAccuracy | 0      | train
5

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val/acc_epoch improved. New best score: 0.930
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/acc_epoch improved. New best score: 0.930


...

INFO: Metric val/acc_epoch improved by 0.016 >= min_delta = 0.0. New best score: 0.946
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/acc_epoch improved by 0.016 >= min_delta = 0.0. New best score: 0.946


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val/acc_epoch did not improve in the last 3 records. Best score: 0.946. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric val/acc_epoch did not improve in the last 3 records. Best score: 0.946. Signaling Trainer to stop.


In [ ]:

modelms2 = Classifier(netMS, weight_decay=0.01, learning_rate=0.00001)
trainer2 = Trainer(seed)

trainer2.fit(modelms2, train_dataloaders=dataloader_trainMS, val_dataloaders=dataloader_valMS)

INFO: Seed set to 1234
INFO:lightning.fabric.utilities.seed:Seed set to 1234
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | ResNet             | 11.2 M | train
1 | loss_fn        | CrossEntropyLoss   | 0      | train
2 | train_accuracy | MulticlassAccuracy | 0      | train
3 | train_loss     | MeanMetric         | 0      | train
4 | val_accuracy   | MulticlassAccuracy | 0      | train
5

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Metric val/acc_epoch improved. New best score: 0.965
INFO:lightning.pytorch.callbacks.early_stopping:Metric val/acc_epoch improved. New best score: 0.965


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: Monitored metric val/acc_epoch did not improve in the last 3 records. Best score: 0.965. Signaling Trainer to stop.
INFO:lightning.pytorch.callbacks.early_stopping:Monitored metric val/acc_epoch did not improve in the last 3 records. Best score: 0.965. Signaling Trainer to stop.
